In [1]:
import osmnx as ox
import pandas as pd
import numpy as np
from Data import Data 
import networkx as nx
from networkx import algorithms 
import math
from numpy import arctan2, sin, cos, sqrt, radians

In [2]:
graph = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', network_type='drive')  

In [3]:
df = Data('sample_larger.csv').df

In [ ]:
#get the nearest edge and node for each location:
nodes = np.zeros(len(df))
EdgeStart = np.zeros(len(df))
EdgeEnd = np.zeros(len(df))
for i in range(len(df)):
    tup1 = (df.iloc[i]['lat'], df.iloc[i]['lon'])
    nodes[i] = ox.get_nearest_node(graph, tup1 , method='euclidean')
    start, end, c = ox.get_nearest_edge(graph, tup1)
    EdgeStart[i] = start
    EdgeEnd[i] = end

In [ ]:
def __construct_graph_data_cols(graph):
    def aux(row):
        coord = (row['lat'],row['lon'])
        nn = ox.get_nearest_node(graph, coord, method='euclidean')
        start, end, _ = ox.get_nearest_edge(graph, coord)
        edge_prog = __edge_progress(graph, start, end, coord)
        start_lon = graph.nodes[start]['x']
        start_lat = graph.nodes[start]['y']
        end_lon = graph.nodes[end]['x']
        end_lat = graph.nodes[end]['y']
        return nn, start, start_lon, start_lat, end, end_lon, end_lat, edge_prog
    return aux

In [ ]:
df['nearest_node'],
    df['nearest_edge_start_node'],
    df['nearest_edge_start_node_lon'],
    df['nearest_edge_start_node_lat'],
    df['nearest_edge_end_node'],
    df['nearest_edge_end_node_lon'],
    df['nearest_edge_end_node_lat'],
    df['edge_progress'] =zip(*df.apply(__construct_graph_data_cols(graph), axis=1))

In [ ]:
def Distance_achieved(edge_node1,edge_node2,lonV, latV):
    #assuming that edge_node1 is the starting point
    lon0 = graph.nodes[edge_node1]['x']
    lon1 = graph.nodes[edge_node2]['x']
    lat0 = graph.nodes[edge_node1]['y']
    lat1 = graph.nodes[edge_node2]['y']
    a = Distance_Euc(lon0, lat0, lon1, lat1)
    b = Distance_Euc(lon0, lat0, lonV, latV)
    return b/a

In [ ]:
def Distance_Euc(lon0, lat0, lon1, lat1):
    lon0 = math.radians(lon0)
    lat0 = math.radians(lat0)
    lon1 = math.radians(lon1)
    lat1 = math.radians(lat1)
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = math.sin(dlat / 2)**2 + math.cos(lat0) * math.cos(lat1) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return 6373.0*c

In [ ]:
def __edge_progress(graph, edge_start_node, edge_end_node, v_coord):
    start_coord = graph.nodes[edge_start_node]['y'], graph.nodes[edge_start_node]['x']
    end_coord = graph.nodes[edge_end_node]['y'], graph.nodes[edge_end_node]['x']

    a = __euc_dist(start_coord, end_coord)
    b = __euc_dist(start_coord, v_coord)
    return b/a


def __euc_dist(coord0, coord1):
    EARTH_RADIUS = 6373

    lat0, lon0 = coord0
    lat1, lon1 = coord1

    lat0, lon0 = radians(lat0), radians(lon0)
    lat1, lon1 = radians(lat1), radians(lon1)

    dlat = lat1 - lat0
    dlon = lon1 - lon0

    a = sin(dlat/2)**2 + cos(lat0) * cos(lat1) * sin(dlon/2)**2
    c = 2 * arctan2(sqrt(a), sqrt(1-a))
    return EARTH_RADIUS*c

In [ ]:
df